In [2]:
import sqlite3
import pandas as pd
products=pd.read_excel("sales_original.xlsx",
                        sheet_name="products")
stock=pd.read_excel("sales_original.xlsx",
                        sheet_name="stock")
assessment=pd.read_excel("sales_original.xlsx",
                        sheet_name="assessment")
invoices=pd.read_excel("sales_original.xlsx",
                        sheet_name="invoices")
customers=pd.read_excel("sales_original.xlsx",
                        sheet_name="customers")                                                
conn=sqlite3.connect("df.db")
c=conn.cursor()

In [3]:
products1=products.drop_duplicates(subset="ASIN")
products1=products1.dropna()
products1=products1[products1["ASIN"]!="Null"]
products1=products1.groupby(["ASIN","product_type"]).sum()
products1=products1.reset_index()
stock1=stock.drop_duplicates(subset="ASIN")
stock1=stock1.dropna()
stock1=stock1[stock1["ASIN"]!="Null"]
assessment1=assessment.drop_duplicates(subset="ASIN")
assessment1=assessment1.dropna()
assessment1=assessment1[assessment1["ASIN"]!="Null"]
invoices1=invoices.drop_duplicates()
invoices1=invoices1[invoices1["ASIN"]!="Null"]
customers1=customers.drop_duplicates(subset="CustomerID")
customers1=customers1.dropna()
customers1=customers1[customers1["CustomerID"]!="Null"]

In [32]:
temp=pd.merge(products1,assessment1,on="ASIN")
temp.to_csv("products.csv")
temp1=pd.merge(invoices1,customers1,on="CustomerID")
temp1.to_csv("sales.csv")

In [ ]:
c.execute("""CREATE TABLE IF NOT EXISTS products
        (ASIN PRIMARY KEY,
        title,
        product_type)
        """)
products1.to_sql("products", conn, if_exists="replace", index=False)
c.execute("""CREATE TABLE IF NOT EXISTS stock
        (StockCode,
        ASIN PRIMARY KEY)
        """)
stock1.to_sql("stock", conn, if_exists="replace", index=False)
c.execute("""CREATE TABLE IF NOT EXISTS assessment
        (rating,
        review_count,
        ASIN PRIMARY KEY)
        """)
assessment1.to_sql("assessment", conn, if_exists="replace", index=False)
c.execute("""CREATE TABLE IF NOT EXISTS invoices
        (InvoiceNo,
        ASIN PRIMARY KEY,
        Quantity,
        price,
        total_sale,
        invoice_date,
        invoice_time,
        CustomerID)
        """)
invoices1.to_sql("invoices", conn, if_exists="replace", index=False)
c.execute("""CREATE TABLE IF NOT EXISTS customers
        (CustomerID
        Country)
        """)
customers1.to_sql("customers", conn, if_exists="replace", index=True)

In [14]:
# Average ratings for each product type
c.execute("""SELECT p.product_type, ROUND(AVG(a.rating),2) 
        FROM products AS p
        JOIN assessment AS a
        ON a.ASIN=p.ASIN
        GROUP BY p.product_type
        ORDER BY AVG(a.rating) DESC
        """).fetchall()

[('mouse', 4.36),
 ('dslr camera', 4.34),
 ('monitor', 4.29),
 ('keyboard', 4.27),
 ('processor', 4.23),
 ('smartphone', 4.13)]

In [23]:
# 5 countries generating max revenue, excluding host
c.execute("""SELECT c.country, ROUND(SUM(i.total_sale))
        FROM customers AS c
        JOIN invoices AS i
        ON i.CustomerID=c.CustomerID
        WHERE c.country IS NOT "Germany"
        GROUP BY c.country
        ORDER BY SUM(i.total_sale) DESC
        LIMIT 5
        """).fetchall()

[('Netherlands', 21692305.0),
 ('Ireland', 15002590.0),
 ('United Kingdom', 12121440.0),
 ('France', 11287137.0),
 ('Australia', 9758509.0)]

In [16]:
# 3 products in each segment with the highest rating
c.execute("""WITH cte AS (
        SELECT * 
        FROM products AS p
        JOIN assessment AS a
        ON a.ASIN=p.ASIN)
        SELECT DISTINCT title, rating, product_type
        FROM
        (
            SELECT cte.*, ROW_NUMBER() OVER (
                PARTITION BY product_type ORDER BY rating DESC) rnk
            FROM cte)
        WHERE rnk<=3
        ORDER BY product_type, rating DESC
        """).fetchall()

[('EN-EL15 EN EL15a Camera Battery Charger Set, YXWIN Replacement Batteries Compatible with Nikon d750, d7200, d7500, d850, d7100, d610, d500, MH-25a, d7000, z6, d810 (2-Pack, Micro USB & Type-C Ports)',
  4.9,
  'dslr camera'),
 ('Pelican 1300 Camera Case With Foam (Yellow)', 4.8, 'dslr camera'),
 ('Pelican 1520 Case With Foam (Black)', 4.8, 'dslr camera'),
 ('Comidox Ergonomic Mousepad with Wrist Support Silicone Gel Wrist Rest Non-Slip Rubber Base Design Protect Your Wrists and De-Clutter Your Desk(Tropical Green Leaf',
  4.9,
  'keyboard'),
 ('Reflex Lab Large Extended Gaming Mouse Pad Mat XXL, Stitched Edges, Waterproof, Ultra Thick 5mm, Wide & Long Mousepad 36â€\x9dx12â€\x9dx.20" Red',
  4.8,
  'keyboard'),
 ('havit Large Gaming Mouse Pad (35.43 X 15.75X 0.12inch) Extended Ergonomic for Computers Thick Keyboard Mouse Mat Non-Slip Rubber Base Mousepad Black',
  4.8,
  'keyboard'),
 ('Portable Monitor, Corprit 13.3" Portable USB C Monitor with Type-c/HDMI/DP Input, 1920Ã—1080 FHD I

In [17]:
# Check if the above items are also the most reviewed
# They are not 
c.execute("""WITH cte AS (
        SELECT * 
        FROM products AS p
        JOIN assessment AS a
        ON a.ASIN=p.ASIN)
        SELECT DISTINCT title, review_count, product_type
        FROM
        (
            SELECT cte.*, ROW_NUMBER() OVER (
            PARTITION BY product_type ORDER BY review_count DESC) rnk
            FROM cte)
        WHERE rnk<=3
        ORDER BY product_type, review_count DESC
        """).fetchall()

[('SanDisk Extreme 32GB MicroSDHC UHS-3 Card - SDSQXAF-032G-GN6MA',
  41907,
  'dslr camera'),
 ('AmazonBasics 60-Inch Lightweight Tripod with Bag', 27293, 'dslr camera'),
 ('AmazonBasics Lightweight Camera Mount Tripod Stand With Bag - 16.5 - 50 Inches',
  18574,
  'dslr camera'),
 ('SanDisk Ultra 32GB microSDHC UHS-I Card with Adapter, Silver, Standard Packaging (SDSQUNC-032G-GN6MA)',
  63885,
  'keyboard'),
 ('Samsung 860 EVO 500GB 2.5 Inch SATA III Internal SSD (MZ-76E500B/AM)',
  21309,
  'keyboard'),
 ('AmazonBasics Wireless Computer Mouse with USB Nano Receiver - Black',
  20270,
  'keyboard'),
 ('ASUS VS207T-P 19.5" HD+ 1600x900 DVI VGA Back-lit LED Monitor,Black',
  5518,
  'monitor'),
 ('AOC e1659Fwu 15.6-Inch Ultra Slim 1366x768 Res 200 cd/m2 Brightness USB 3.0-Powered Portable LED Monitor w/ Case',
  3443,
  'monitor'),
 ('ASUS ROG Swift PG248Q 24â€\x9d Gaming Monitor Full HD 1080p 1ms 180Hz DP HDMI Eye Care G-SYNC eSports',
  3270,
  'monitor'),
 ('VicTsing mm057 2.4G Wire

In [18]:
# 3 best selling products in each category
c.execute("""WITH cte1 AS (
        SELECT ASIN, SUM(Quantity) AS sold
        FROM invoices AS i
        GROUP BY ASIN),
        cte2 AS (
            SELECT p.product_type, cte1.ASIN, p.title,
            RANK () OVER (PARTITION by product_type ORDER BY sold DESC) rnk
            FROM cte1
            LEFT JOIN products AS p
            ON p.ASIN=cte1.ASIN)
        SELECT * 
        FROM cte2
        WHERE rnk BETWEEN 0 AND 3
        """).fetchall()

[('dslr camera',
  'B00D76RNLS',
  'Manfrotto PIXI Mini Tripod, Black (MTPIXI-B)',
  1),
 ('dslr camera',
  'B008MWBY6W',
  'AmazonBasics Holster Camera Case for DSLR Cameras - 7 x 6 x 9 Inches, Black',
  2),
 ('dslr camera',
  'B00CF5OHZ2',
  'AmazonBasics Large DSLR Camera Gadget Bag - 11.5 x 6 x 8 Inches, Black And Orange',
  3),
 ('keyboard',
  'B07KG95L9H',
  'MOSISO MacBook Air 13 inch Case 2020 2019 2018 Release A2179 A1932, Plastic Peony Hard Shell&Keyboard Cover&Screen Protector Only Compatible with MacBook Air 13 inch with Retina Display, Black',
  1),
 ('keyboard',
  'B010Q57T02',
  'SanDisk Ultra 32GB microSDHC UHS-I Card with Adapter, Silver, Standard Packaging (SDSQUNC-032G-GN6MA)',
  2),
 ('keyboard',
  'B07Y693ND1',
  'Razer DeathAdder Essential Gaming Mouse: 6400 DPI Optical Sensor - 5 Programmable Buttons - Mechanical Switches - Rubber Side Grips - White',
  3),
 ('monitor',
  'B01M4II4M4',
  'ASUS ROG Swift PG248Q 24â€\x9d Gaming Monitor Full HD 1080p 1ms 180Hz DP HD

In [19]:
# Get the first and second worst selling products in each category
# Multiple items have sale quantity 1, so barring any other filters,
# we can not select only 2 for each category
c.execute("""WITH cte1 AS (
        SELECT ASIN, SUM(Quantity) AS sold
        FROM invoices AS i
        GROUP BY ASIN),
        cte2 AS (
            SELECT p.product_type, cte1.ASIN, p.title,
            RANK () OVER (PARTITION by product_type ORDER BY sold) rnk
            FROM cte1
            LEFT JOIN products AS p
            ON p.ASIN=cte1.ASIN)
        SELECT * 
        FROM cte2
        WHERE rnk BETWEEN 0 AND 2
        ORDER BY product_type
        """).fetchall()

[('dslr camera', 'B002JB07D8', 'Pelican Storm iM2975 Case No Foam (Black)', 1),
 ('dslr camera',
  'B008SM1EWU',
  'Nikon D7200 GPS Receiver, Micnova GPS-N Plus DSLR Camera GPS Unit Navigation Geotagging for Nikon D800 D3200 D3300 D90 D7100 D5200 D4 D600 D5100 D7000 D300 D300S',
  1),
 ('dslr camera',
  'B00N1B483K',
  'Rokinon Cine DS 50mm T1.5 AS IF UMC Full Frame Cine Lens for Nikon',
  1),
 ('dslr camera',
  'B018I844Z2',
  'Neewer Rubber-Coated Metal Camera Lens Filter Remover Wrench Set Kit(Package of Two), Fit 58mm,62mm Lens Thread for Canon,Nikon,Sony,Pentax,Fujifilm,Olympus,Panasonic and Other DSLR Cameras',
  1),
 ('dslr camera',
  'B01DFHO4OM',
  'Sigma 17-50mm f/2.8 EX DC OS HSM FLD Large Aperture Standard Zoom Lens for Canon Digital DSLR Camera - International Version (No Warranty)',
  1),
 ('dslr camera',
  'B01E8I6E8Q',
  'YONGNUO YN685 GN60 2.4G System ITTL HSS Wireless Flash Speedlite with Radio Slave for Nikon DSLR Cameras',
  1),
 ('dslr camera',
  'B01FIGK6ZO',
  'K

In [20]:
# Find the unique customers per month
c.execute("""WITH cte AS
                (SELECT DISTINCT CustomerID, 
                strftime("%m",invoice_date) AS month
                FROM invoices
                WHERE strftime("%Y",invoice_date)="2019")
        SELECT COUNT(c.CustomerID), month
        FROM cte
        JOIN Customers AS c
        ON cte.CustomerID=c.CustomerID
        GROUP BY month
        """).fetchall()

[(741, '01'),
 (758, '02'),
 (974, '03'),
 (851, '04'),
 (1056, '05'),
 (991, '06'),
 (948, '07'),
 (934, '08'),
 (1263, '09'),
 (1363, '10'),
 (1664, '11'),
 (615, '12')]